<a href="https://colab.research.google.com/github/Oriolrt/MVC_M2_PGM/blob/main/2_BN2FG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conversion to a Factor Graph

In this example we will show how to convert a Bayesian Network to a Factor graph using the pgmpy package (https://pgmpy.org/). As we can apply exact inference algorithms in both networks, we'll check that we obtain the same results in both models.

First, we install the pgmpy package and import the needed classes and methods

In [ ]:
# @title
!pip install pgmpy
#!apt update
#!apt install imagemagick
#!apt install pdf2svg
#!apt install texlive texlive-latex-extra
#!pip install git+git://github.com/mkrphys/ipython-tikzmagic.git
#%load_ext tikzmagic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00


In [ ]:
# Importing Library
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.metrics.bn_inference import BayesianModelProbability
from pgmpy.inference import BeliefPropagation


The Bayesian network model is the one shown in the figure below:

<img src='https://drive.google.com/uc?id=1VHH4MV8d150FQcwbzL9Maq-cwa6KGTOM' height='400' >

The pgmpy code to built it is found in tyhe cell below:

In [ ]:
# Defining network structure

alarm_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "NeighborCalls"),
    ]
)

# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.7], [0.3]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.9], [0.1]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.99, 0.1, 0.3, 0.01], [0.01, 0.9, 0.7, 0.99]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_neighborcalls = TabularCPD(
    variable="NeighborCalls",
    variable_card=2,
    values=[[0.9, 0.2], [0.1, 0.8]],
    evidence=["Alarm"],
    evidence_card=[2],
    state_names={"NeighborCalls":["no","yes"],"Alarm":[False,True]},
)

# Associating the parameters with the model structure
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_neighborcalls
)

The factor graph is defined


To convert a BN to a Factor graph we have, first, to __marry__ all the parents of child nodes (__moralization step__). This step can easily be done as shown below

In [ ]:
from pgmpy.models import FactorGraph
import numpy as np

# Viewing nodes and edges of the model

nodes =alarm_model.nodes()
edges = np.array(alarm_model.edges())

print('BN edges: \n', edges)

#Identifying the nodes to be married (Moralizing)
#childs= np.unique(edges[:,1])
#factor_nodes = []
#for ch in childs:
#  factor_nodes = factor_nodes + [list(edges[edges[:,1]==ch,0]) + [ch]]

#print( factor_nodes)

# Alternatively, in a more compacted way
factor_nodes = [ list(edges[edges[:,1]==ch,0]) + [ch] for ch in np.unique(edges[:,1])]
print('\n factor nodes: \n', factor_nodes)

BN edges: 
 [['Burglary' 'Alarm']
 ['Alarm' 'NeighborCalls']
 ['Earthquake' 'Alarm']]

 factor nodes: 
 [['Burglary', 'Earthquake', 'Alarm'], ['Alarm', 'NeighborCalls']]


The __factor_nodes__ list shows that we will have 2 factor functions: $\phi_1$ and $\phi_2$. The scope of the first factor function $\phi_1$ is given by variables: 'Burglary', 'Earthquake' and  'Alarm', while for $\phi_2$ is given by variables: 'Alarm' and 'NeighborCalls'.

Considering the CPDs in the BN model, the factor functions are:
\begin{align}
\phi_1 &= P(A|E,B)P(E)P(B) = P(\text{'Alarm'}| \text{'Earthquake'}, \text{'Burglary'})P(\text{'Earthquake'})P( \text{'Burglary'}) \\
\phi_2 &= P(N|A) = P(\text{'NeighborCalls'}| \text{'Alarm'})
\end{align}

which means that we have a factor of order 3 ($\phi_1$) and a factor of order 2 ($\phi_2$).

In [ ]:
for  cpd in alarm_model.get_cpds():
  print(cpd)
  print(cpd.cardinality)
  print(cpd.values)

+-------------+-----+
| Burglary(0) | 0.7 |
+-------------+-----+
| Burglary(1) | 0.3 |
+-------------+-----+
[2]
[0.7 0.3]
+---------------+-----+
| Earthquake(0) | 0.9 |
+---------------+-----+
| Earthquake(1) | 0.1 |
+---------------+-----+
[2]
[0.9 0.1]
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.99          | 0.1           | 0.3           | 0.01          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.01          | 0.9           | 0.7           | 0.99          |
+------------+---------------+---------------+---------------+---------------+
[2 2 2]
[[[0.99 0.1 ]
  [0.3  0

Unfortunately, the __pgmpy__ package does not provide a way to multiply the CPD in a way we define factors $\phi_1$ and $\phi_2$ as defined above. However, we can build an equivalent *factor graph* structure in pgm that will allow to us to perform inference on it.

The way to proceed is defining a factor function for each CPD as shown below. The loop iterates on the CPDs of the BN model. For each CPD of the BN we build a factor function with the same variables, cardinality, values and state_names and we add it to the factor graph. Then we add edges from variables to the factor (remember that a factor graph is a bivariate graph composed by factor functions and random variables).

In [ ]:
from pgmpy.factors.discrete.CPD import DiscreteFactor
G = FactorGraph()
G.add_nodes_from(alarm_model.nodes())
f_nodes = factor_nodes[0]
for  cpd in alarm_model.get_cpds():
  phi = DiscreteFactor(cpd.variables,cardinality=cpd.cardinality,values=cpd.values,state_names=cpd.state_names)
  G.add_factors(phi)
  G.add_edges_from([(v,phi) for v in cpd.variables])


print(G)
G.get_partition_function()

FactorGraph with 8 nodes and 7 edges


1.0

The 8 nodes of the factor graph correspond to the 4 CPDs and the 4 random variables. Moreover, as the joint distribution of the BN, and the factor graph, is given in both models by the CPDs, the __partition function__ is 1.

# Inference on the Factor Graph

pgmpy provides some inference algorithms for Bayesian networks and other Probabilistic graphical models

In [ ]:
from pgmpy.inference import BeliefPropagation

bp_infer = BeliefPropagation(G)
bp_infer.get_cliques()

bp_infer.calibrate()

In [ ]:
q=bp_infer.query(variables=["Burglary"], evidence={"NeighborCalls": "yes"})
print(q)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.3929 |
+-------------+-----------------+
| Burglary(1) |          0.6071 |
+-------------+-----------------+


In [ ]:
q=bp_infer.map_query(variables=["Burglary"], evidence={"NeighborCalls": "yes"}, show_progress=True)
print(q)

{'Burglary': 1}


## A deeper look....

The __BeliefPropagation__ class above converts Bayesian networks, factor graphs and othe PGMs to a junction tree needed to apply belief propagation.

Below we can see the cliques obtained after building the junction trees of the BN model and its factor graph:

In [ ]:
alarm_infer = BeliefPropagation(alarm_model)
alarm_infer.get_cliques()


NodeView((('Alarm', 'NeighborCalls'), ('Alarm', 'Burglary', 'Earthquake')))

In [ ]:
bp_infer = BeliefPropagation(G)
bp_infer.get_cliques()


NodeView((('Alarm', 'NeighborCalls'), ('Alarm', 'Burglary', 'Earthquake')))

We can see that in both cases we obtain the same cliques, as expected.

Below we get the clique beliefs obtained from the Factor Graph __G__ and the BN model after calibrating them.

In [ ]:
alarm_infer.calibrate()
bp_infer.calibrate()

cbn=alarm_infer.get_clique_beliefs()
cb= bp_infer.get_clique_beliefs()
print(cbn)
print(cb)

{('Alarm', 'NeighborCalls'): <DiscreteFactor representing phi(NeighborCalls:2, Alarm:2) at 0x7d7bf59685b0>, ('Alarm', 'Burglary', 'Earthquake'): <DiscreteFactor representing phi(Burglary:2, Earthquake:2, Alarm:2) at 0x7d7bf596bd30>}
{('Alarm', 'NeighborCalls'): <DiscreteFactor representing phi(NeighborCalls:2, Alarm:2) at 0x7d7bf596be50>, ('Alarm', 'Burglary', 'Earthquake'): <DiscreteFactor representing phi(Burglary:2, Earthquake:2, Alarm:2) at 0x7d7bf59690c0>}


It can be seen the that in both cases we obtain exactly the same value

In [ ]:
for var,cpds in cb.items():
  print(var)
  # print(cpds.values)
  # print(cbn[var].values)
  print(cpds.values - cbn[var].values)

('Alarm', 'NeighborCalls')
[[0. 0.]
 [0. 0.]]
('Alarm', 'Burglary', 'Earthquake')
[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]


## Exercise
Compute the 3 order factor function $\phi_1$ from the list of CPDs of the BN. Consider the lexical order to sort the variables

In [ ]:

# lexical order
vars = ('Alarm', 'Burglary', 'Earthquake')

# This means that Alarm will be dim=0, Burglary dim=1 and Earthquake dim=2

# we have to reshape the CPDs values in a 3 dimensional numpy array according to the variables sorting defined above
#cpd_burglary, cpd_earthquake, cpd_alarm,
print(cpd_burglary.values)

# TODO: Replace the line above by the numpy array of cpd_burglary.values with dimensions (1,2,1)
B = np.ones((1,2,1))
print(B)

# TODO: Replace the line above by the numpy array of cpd_earthquake.values with dimensions (1,2,1)
E = np.ones((1,1,2))
print(B)

# TODO
# The cpd_alarm.values are given following the order that appear in the cpd_alarm.variables.
# The order of the variables in cpd_alarm.variables must be compared to the order
# in the vars dictionary. If both orders are note equal, the cpd_alarm.values dimensions must be permuted accordingly.

ABE = np.ones((2,2,2))

phi = ABE*B*E

print(phi)


[0.7 0.3]
[[[1.]
  [1.]]]
[[[1.]
  [1.]]]
[[[1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]]]


and we build a *new* version of the factor graph using the 3-order factor computed above

In [ ]:
G1 = FactorGraph()
G1.add_nodes_from(alarm_model.nodes())

# phi_1 factor (Computed in the cell above)
#TODO: uncomment the last line below and complete the constructor of the Discrete Factor
# You must identify the following parameters:
# - variables: the vars dictionary defined above.
# - cardinality: the same than in the cpd_alarm
# - values: the ones computed above
# - state_names: the same than in the cpd_alarm
#
# phi = DiscreteFactor( )

G1.add_factors(phi)
G1.add_edges_from([(v,phi) for v in vars])


# phi_2 factor = neighborcalls factor
#TODO: uncomment the last line below and complete the constructor of the Discrete Factor.
# As before you must identify the same parameters but in this case you can take the ones in cpd_neighborcalls
#
# phi = DiscreteFactor( )

G1.add_factors(phi)
G1.add_edges_from([(v,phi) for v in cpd_neighborcalls.variables])


print(G1)
G1.get_partition_function()

AttributeError: 'numpy.ndarray' object has no attribute 'variables'

We build a new BeliefProgation object and infer the P( Burglary | NeighborCalls= "yes" ) to compare with the previous results

In [ ]:
bp1_infer = BeliefPropagation(G1)
bp1_infer.get_cliques()

bp1_infer.calibrate()
q1=bp1_infer.query(variables=["Burglary"], evidence={"NeighborCalls": "yes"})
print(q1)